# Coreloss Example Extended
In this notebook, a typical workflow of EELS processing is shown on simulated data. This notebook shows one example of how core-loss quantification could be performed. This notebook is very similar to the *CorelossExample* notebook with the difference being in the simulation of the multispectrum which is explained in this notebook.

In [1]:
%matplotlib qt 
#important for the em.MultiSpectrumVisualizer since this is an interactive plotting tool

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pyEELSMODEL.api as em

## Simulation core-loss
In this section , the core-loss signal will be simulated. This step is in general not needed since we get the experimental data instead of simulating it.



In [3]:
#functions to make easy mask to make elemental maps
def make_circular_mask(xco, yco, Rin, Rout, shape):
    XX, YY = np.meshgrid(np.arange(shape[1])-yco, np.arange(shape[0])-xco)
    R = np.sqrt(XX**2+YY**2)
    
    mask = np.zeros(shape)
    boolean = (R>=Rin) & (R<Rout)
    mask[boolean] = 1
    return mask

def make_rectangular_mask(xco, yco, width,height, shape):
    mask = np.zeros(shape)
    mask[xco:xco+width, yco:yco+height] = 1
    return mask

#### Elemental abundance maps
Define the elements and the eges used. Add the atomic numbers since they will be used to have some estimate on the free mean path. \
Different masks are defined to showcase an elemental abundance at each position

In [4]:
elements = ['C', 'N', 'O', 'Fe']
edges = ['K', 'K', 'K', 'L']
Zs = [6, 7, 8, 26] #atomic weights

#scan size elemental map
xsize = 128
ysize = 128
maps = np.zeros((len(elements),xsize,ysize))

#masks which define different regions. 
mask0 =make_rectangular_mask(5, 5, 20, 20, (xsize,ysize))
mask1 =  make_rectangular_mask(90, 90, 20, 30, (xsize,ysize))
mask2 = make_circular_mask(xsize//2, ysize//2, 20, 30, (xsize,ysize))
mask3 = make_circular_mask(xsize//2, ysize//2, 0, 20, (xsize,ysize))

#attribute different elemental values for different masks
maps[0] = 1  #carbon elemental map
maps[1] = 2*mask0 + mask1 #nitrogen elemental map
maps[2] = mask2 #oxygen elemental map
maps[3] = mask3+0.5*mask2 #iron elemental map

In [5]:
#shows the real truth elemental abundance
fig, ax = plt.subplots(1,len(elements))
for ii in range(maps.shape[0]):
    ax[ii].imshow(maps[ii], cmap='gray')
    ax[ii].set_title(elements[ii])

#### t/&lambda;
Calculate some measure of inelastic mean free path to include this into the simulated data. Since a change in inelastic mean free path modfies the multiple scattering which then changes the shape of background and core loss edge

In [6]:
adf = np.zeros((xsize, ysize))
tlambda_map = np.zeros_like(adf)
for ii in range(maps.shape[0]):
    adf += (Zs[ii]*maps[ii])**2
    tlambda_map += Zs[ii]*maps[ii]
    
tlambda_map = tlambda_map/tlambda_map.max() #maximum t/lambda is 1 for this sample

In [7]:
#shows adf and mean-free path figure
fig, ax = plt.subplots(1,2)
ax[0].imshow(adf, cmap='gray')
ax[0].set_title(r'ADF like contrast')
ax[1].imshow(tlambda_map, cmap='gray')
ax[1].set_title('mean free path')

Text(0.5, 1.0, 'mean free path')

#### Simulation part
CoreLossSimulator is an object which simulates a core loss multispectrum from the given elements, maps, t/&lambda; and the settings.\
Here is a small description on how the multispectrum gets simulated:
1. Calculate a low loss which contains a zero-loss (Gaussian) with a FWHM (1eV) and the low loss. The low loss only uses the bulk plasmons, these are calculated using the plasmon energy (22 eV) and t/&lambda;. In this simulation the only variying parameter in the low loss is the t/&lambda; . This is done to showcase the influence of multiple scattering on the core-loss spectra.
2. The background is calculated by starting with a powerlaw (A E<sup>-3</sup>) which gets convolved with the low loss at each probe position.
3. The core loss edges are calculated using the E0, alpha, beta and GOS tables available. The calculated shape then gets convolved with the low loss to thake the multiple scattering into account. The GOS tables used are the one calculated by [Zhang Zezhong](https://zenodo.org/records/7729585)
4. Poisson noise is added to the core-loss.
5. To emulate the effect of instabilities in the spectrometer, both the low loss and core loss are shifted at each probe position.

In [8]:
E0 = 300e3 #acceleration voltage [V]
alpha = 1e-9 #convergence angle [rad]
beta = 20e-3 #collection angle [rad]

dispersion = 0.5 #[eV]
offset = 200 #[eV]
size = 2048 #number of pixel used in simulation [eV]

settings = (E0, alpha, beta)
msh = em.MultiSpectrumshape(dispersion, offset, size, xsize, ysize)


In [9]:
sim = em.CoreLossSimulator(msh, elements, edges, maps, tlambda_map, settings)
sim.use_shift = True #add shift which experimentally arises due to instabilities
sim.simulate_multispectrum()

16384it [00:12, 1278.41it/s]
16384it [00:37, 431.62it/s]
16384it [00:00, 34154.28it/s]


Multispectrum is simulated


In [11]:
#redefine the multispectra for easy use
hl = sim.multispectrum
ll = sim.ll

## Quantification Part
The previous part is the more irrelevant part which simulates the core-loss. In most of the cases, the data is available and needs to be processed. In this part, a typical workflow on getting the elemental abundance together with estimated errors is shown. Note that the ElementalQuantification class has such a workflow implemented in it but here we show how one can design their own workflow to optimize the data processing.

#### Visualization
The first part of processing (EELS) data is by visually inspecting it. Here are some function which are available to visualize the data and do some manual inspection of it.


In [11]:
#show the mean core loss spectrum
hl.mean().plot()

In [12]:
#show a single spectrum at index position nx, ny
nx = 64
ny = 64

hl.setcurrentspectrum((64,64))
hl.plot()

In [13]:
#show the mean low loss spectrum
#note that the drift broadens the zero loss peak
ll.mean().plot()

In [14]:
#visualize the multispectrum
#use + to increase box size --> take average inside box
#use - to descrease box size --> takes average inside box
#arrows can be used to navigate 
#clicking and dragging also work to navigate

em.MultiSpectrumVisualizer([hl])

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


#### Aligning multispectra
The core-loss gets aligned using the low-loss. Multiple methods are available for find the appropriate shifts and correct for it.
1. FastAlignZeroLoss: Uses the energy at which the maximum intensity is measured. The found shifts are applied by rolling each spectra to make it align. This method is fast and does not modify the inputted data via interpolation. Hence it cannot find subpixel positions. This method works best when the zero-loss peak is sharp and has a high intensity which is valid in most cases. In our experience, this method works really well for elemental quantification.
2. AlignZeroLoss: Fits a model to the zero-loss peak where the model is a Gaussian or Lorentzian function which needs to be specified by the user. This method is a lot slower and can be unstable due to its non-linear fitting procedure but has the potential to correct for subpixel shifts and works for a noisy and not sharp zero-loss peak.
3. AlignCrossCorrelation: Finds the shift which gives the best similarity between two spectra by cross correlation the two spectra. Subpixel accuracy can be obtained via interpolating the experimental data and finding the shift. This method is generally faster than the AlignZeroLoss but could fail if the low loss spectra are not very similar to each other. This method can also be used to align core-loss signal when no low-loss is available. 



In [15]:
fast_align = em.FastAlignZeroLoss(ll, other_spectra=[hl], cropping=True)
align = em.AlignZeroLoss(ll, other_spectra=[hl], cropping=True)
cros_align = em.AlignCrossCorrelation(ll, other_spectra=[hl], cropping=True, is_zlp=True)

In [16]:
print('Start using FastAlignZeroLoss object')
fast_align.perform_alignment()
print('Start using AlignZeroLoss object')
align.perform_alignment()
print('Start using AlignCrossCorrelation object')
cros_align.perform_alignment()

Start using FastAlignZeroLoss object
Start using AlignZeroLoss object
Estimates the parameters for the fitting procedure


16384it [00:45, 357.26it/s]
16384it [00:03, 4125.13it/s]


Start using AlignCrossCorrelation object


16384it [00:08, 2034.76it/s]
16384it [00:03, 4327.35it/s]


In [18]:
fig = fast_align.show_shift()
fig = align.show_shift()
fig = cros_align.show_shift()

In [19]:
#small comparison plot between the different alignement methods. 
#there should be almost no difference between them
#shows how to manipulate the data and visualize it via matplotlib 
fig, ax = plt.subplots(1,2)
ax[0].plot(fast_align.aligned.energy_axis, fast_align.aligned.mean().data, label='FastAlignZeroLoss') 
ax[0].plot(align.aligned.energy_axis, align.aligned.mean().data, label='AlignZeroLoss') 
ax[0].plot(fast_align.aligned.energy_axis, fast_align.aligned.mean().data, label='AlignCrossCorrelation') 
ax[0].set_title(r'Low loss')
ax[0].legend()

ax[1].plot(fast_align.aligned_others[0].energy_axis, fast_align.aligned_others[0].mean().data, label='FastAlignZeroLoss') 
ax[1].plot(align.aligned_others[0].energy_axis, align.aligned_others[0].mean().data, label='AlignZeroLoss') 
ax[1].plot(fast_align.aligned_others[0].energy_axis, fast_align.aligned_others[0].mean().data, label='AlignCrossCorrelation') 
ax[1].set_title(r'Core loss')

Text(0.5, 1.0, 'Core loss')

In [20]:
hl_al = fast_align.aligned_others[0] #coreloss which is used for quantification
ll_al = fast_align.aligned #lowloss which is used for quantification

#### Define the model
The next step is to chose a proper model for the experimental data. In pyEELMODEL each model consist out of  components and each component has multiple parameters. For instance, a gaussian component has three parameters: ampltide, center and fwhm. For each parameter one can identify if it needs to be changeable or not. If it is not changeable then it will not be updated via the fitting procedure. The cross sections have multiple parameters but for elemental quantification only the amplitude is the unkown. More information on the model-based approach can be found here [[1]](https://doi.org/10.1016/j.ultramic.2006.05.006) \
The model for this example consists in this case out of three parts:
1. **The background**: Historically a powerlaw was used to model the background but in this example a linear background model is used[[2]](https://doi.org/10.1016/j.ultramic.2023.113830). This keeps the entire model linear which is advantages because no starting parameters are needed and no iterations need to be performed to find the global optimum.
2. **Atomic cross sections**: The generalized oscillator strengths from Zhang et al. [[3]](https://zenodo.org/records/7729585) are used. To properly calculate these cross sections, the acceleration voltage (E0), convergence angle (alpha) and collection angle (beta) are needed as input.
3. **The low loss**: Due to multiple scattering, the shape of cross sections will be modified and this can be taken into account if the low loss is acquired from the same area. Note that the background will not be convoluted in the model since this is hard to incorporate due to the artifacts arising from the boundaries [[1]](https://doi.org/10.1016/j.ultramic.2006.05.006).\
\
[1] Verbeeck J. et al; Model based quantification of EELS spectra; Ultramicroscopy; 2004; doi:[10.1016/j.ultramic.2006.05.006](https://doi.org/10.1016/j.ultramic.2006.05.006)\
[2] Van den Broek W. et al; Convexity constrains on linear background models for electron energy loss spectra; Ultramicroscopy; 2023; doi:[10.1016/j.ultramic.2023.113830](https://doi.org/10.1016/j.ultramic.2023.113830)\
[3] Zhang Z. et al; Generalised oscillator strngth for core-shell excitation by fast electron based on Dirac solutions; Zenodo; 2023; doi:[10.5281/zenodo.7729585](https://zenodo.org/records/7729585)


##### Background component
The linear combination of fixed powerlaws where the powers are given by rlist:\
$$bg(E) = \sum_{i=0}^n A_i E^{-i}$$

In [47]:
from pyEELSMODEL.components.linear_background import LinearBG

In [48]:
n = 4
bg = LinearBG(specshape=hl_al.get_spectrumshape(), rlist=np.linspace(1,5,n))

##### Cross sections
The cross sections are calculated using the cross sections of [Zezhong Zhang](https://zenodo.org/records/7729585). In pyEELSMODEL, the hydrogenic K and L edges and the cross section from [Segger, Guzzinati and Kohl](https://zenodo.org/records/7645765) are also available. 

In [49]:
from pyEELSMODEL.components.CLedge.zezhong_coreloss_edgecombined import ZezhongCoreLossEdgeCombined
from pyEELSMODEL.components.CLedge.kohl_coreloss_edgecombined import KohlLossEdgeCombined

In [50]:
elements = ['C', 'N', 'O', 'Fe']
edges = ['K', 'K', 'K', 'L']
E0 = 300e3 
alpha = 1e-9
beta = 20e-3 

Showcase the difference between the two different GOS tables. 

In [63]:
#can take a bit of time since the cross section is calculated from the tabulated GOS arrays
fig, ax = plt.subplots(1,len(elements))
for ii in range(len(elements)):
    compz = ZezhongCoreLossEdgeCombined(hl_al.get_spectrumshape(), 1, E0, alpha,beta, elements[ii], edges[ii])
    compz.calculate() #calculates the cross section with given parameters
    compk = KohlLossEdgeCombined(hl_al.get_spectrumshape(), 1, E0, alpha,beta, elements[ii], edges[ii]) 
    compk.calculate()

    ax[ii].plot(compz.energy_axis, compz.data, label='Zhang')
    ax[ii].plot(compk.energy_axis, compk.data, label='Kohl')
ax[0].legend()



L2 is used
L1 is used


In [51]:
#can take a bit of time since the cross section is calculated from the tabulated GOS arrays
#can chose which cross section you use 
comp_elements = []
A = 1 #amplitude for cross section, since model is linear this value is not super important. For non-linear fitters the starting value could be important
for elem, edge in zip(elements, edges):
    #comp = ZezhongCoreLossEdgeCombined(hl_al.get_spectrumshape(), 1, E0, alpha,beta, elem, edge)
    comp = KohlLossEdgeCombined(hl_al.get_spectrumshape(), 1, E0, alpha,beta, elem, edge)
    comp_elements.append(comp)


L2 is used
L1 is used


##### Multiple scattering
The calculated components are convolved with the low loss if indicated. For instance, the background component will not be convolved with the lowloss 

In [52]:
from pyEELSMODEL.components.MScatter.mscatterfft import MscatterFFT

In [53]:
llcomp  = MscatterFFT(hl_al.get_spectrumshape(), ll_al)

##### Model
The model gets created by adding all the different components together into a list. It uses this information to calculate the resulting model and can be used as input for the fitter.

In [54]:
components = [bg]+comp_elements+[llcomp]
mod = em.Model(hl_al.get_spectrumshape(), components)

In [55]:
#shows the model with the given paramter values. 
mod.calculate()
mod.plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


#### Finding optimal parameters for model
Since the model we defined is linear, we can use a weighted linear fitter. The weights are determined from the assumption that the noise is poisson distributed. 

In [56]:
#creates a fit object 
fit = em.LinearFitter(hl_al, mod, use_weights=True)

cannot use analytical gradients since a convolutor is inside model


In [57]:
fit.multi_fit()

16384it [00:38, 421.78it/s]


In [58]:
# the fitted parameters can be found in .coeff_matrix attribute.
print(fit.coeff_matrix.shape)
print(str(fit.coeff_matrix.shape[2])+' parameters are optimized in the fitting procedure')

#To know which parameter corresponds to which index in the coeff_matrix, following function can be used
N_index = fit.get_param_index(comp_elements[1].parameters[0]) #comp_elements[1].parameters[0]: amplitude of nitrogen edge
N_map = fit.coeff_matrix[:,:,N_index]

fig, ax = plt.subplots()
ax.imshow(N_map)
ax.set_title(comp_elements[1].name)

(128, 128, 8)
8 parameters are optimized in the fitting procedure


Text(0.5, 1.0, 'N K edge: 402 eV')

In [59]:
#function shows the elemental maps 
fig, maps, names = fit.show_map_result(comp_elements)

In [40]:
#calculates the fitted model, this can be used to validate visually the fitted results
multimodel = fit.model_to_multispectrum()

16384it [00:11, 1378.26it/s]


In [60]:
em.MultiSpectrumVisualizer([hl_al, multimodel])

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [37]:
#calculates the cramer rao lower bound for the given paramter at each probe position
crlb_Fe = fit.CRLB_map(comp_elements[3].parameters[0]) #comp_elements[3].parameters[0]: amplitude of iron edge

16384it [00:26, 611.58it/s]


In [38]:
fig, ax = plt.subplots()
ax.imshow(crlb_Fe)

### ElementalQuantification class
The last part shows how the ElementalQuantification class is used as workflow to get the same result. The workflow used in this example is similar to what is used in the ElementalQuantification class but this class gives some other attributes with which you can vary the workflow. For more information see the documentation or the other notebook ()

In [12]:
quant = em.ElementalQuantification(hl, elements, edges, settings, ll=ll)
quant.n_bgterms = 4
quant.linear_fitter_method = 'ols'
quant.do_procedure()

cannot use analytical gradients since a convolutor is inside model
16384it [00:37, 442.52it/s]
16384it [00:12, 1320.05it/s]


In [45]:
quant.show_elements_maps()